##LSTM using only close

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
df = yf.download('AAPL')

[*********************100%***********************]  1 of 1 completed


In [ ]:
!curl -L http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -O && tar xzvf ta-lib-0.4.0-src.tar.gz
!cd ta-lib && ./configure --prefix=/usr && make && make install && cd - && pip install ta-lib
import talib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   380  100   380    0     0   6368      0 --:--:-- --:--:-- --:--:--  6440
100   353  100   353    0     0   3066      0 --:--:-- --:--:-- --:--:-- 13074
100 1299k  100 1299k    0     0  1202k      0  0:00:01  0:00:01 --:--:-- 1202k
ta-lib/
ta-lib/config.sub
ta-lib/aclocal.m4
ta-lib/CHANGELOG.TXT
ta-lib/include/
ta-lib/include/ta_abstract.h
ta-lib/include/ta_func.h
ta-lib/include/ta_common.h
ta-lib/include/ta_config.h.in
ta-lib/include/Makefile.am
ta-lib/include/ta_libc.h
ta-lib/include/ta_defs.h
ta-lib/missing
ta-lib/ta-lib.spec.in
ta-lib/config.guess
ta-lib/Makefile.in
ta-lib/ta-lib.dpkg.in
ta-lib/Makefile.am
ta-lib/autogen.sh
ta-lib/install-sh
ta-lib/configure
ta-lib/depcomp
ta-lib/HISTORY.TXT
ta-lib/configure.in
ta-lib/autom4te.cache/
ta-lib/autom4te.cache/output.0
ta-lib/autom4te.cache/requests
ta-lib/autom4te.cache/outpu

In [ ]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,10740.000000,10740.000000,10740.000000,10740.000000,10740.000000,1.074000e+04
mean,18.342701,18.549519,18.144349,18.355745,17.646304,3.242764e+08
std,38.653287,39.107432,38.234216,38.693046,38.336785,3.370176e+08
min,0.049665,0.049665,0.049107,0.049107,0.038102,0.000000e+00
25%,0.291295,0.299107,0.284224,0.291295,0.239871,1.185529e+08
50%,0.497210,0.506651,0.489978,0.497768,0.412027,2.117360e+08
75%,18.001697,18.135625,17.836875,17.950715,15.638111,4.042199e+08
max,195.089996,198.229996,192.649994,195.100006,195.100006,7.421641e+09


In [ ]:
df.index=pd.to_datetime(df.index,format='%Y-%m-%d')
res = df[~(df.index < '2015-01-01')]
print(res)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2015-01-02   27.847500   27.860001   26.837500   27.332500   24.531765   
2015-01-05   27.072500   27.162500   26.352501   26.562500   23.840666   
2015-01-06   26.635000   26.857500   26.157499   26.565001   23.842911   
2015-01-07   26.799999   27.049999   26.674999   26.937500   24.177240   
2015-01-08   27.307501   28.037500   27.174999   27.972500   25.106184   
...                ...         ...         ...         ...         ...   
2023-07-14  190.229996  191.179993  189.630005  190.690002  190.690002   
2023-07-17  191.899994  194.320007  191.809998  193.990005  193.990005   
2023-07-18  193.350006  194.330002  192.419998  193.729996  193.729996   
2023-07-19  193.100006  198.229996  192.649994  195.100006  195.100006   
2023-07-20  195.089996  196.470001  192.500000  193.130005  193.130005   

               Volume  
Date         

In [ ]:
df=res

In [ ]:
df['MA']=talib.SMA(df['Adj Close'],timeperiod=12)
df['EMA']=talib.EMA(df['Adj Close'],timeperiod=12)
EMA_12 = pd.Series(df['Close'].ewm(span=12,min_periods=12).mean())
EMA_26 = pd.Series(df['Close'].ewm(span=26,min_periods=26).mean())
df['MACD']=pd.Series(EMA_26-EMA_12)
df['MACD_Signal']=pd.Series(df['MACD'].ewm(span=9,min_periods=9).mean())
df['RSI'] = talib.RSI(df['Adj Close'], 10)
df['WMA']=talib.WMA(df['Adj Close'],timeperiod=14)
df['Upper Band'],df['Mid Band'],df['Low Band']=talib.BBANDS(df['Adj Close'],timeperiod=50)
df['DI+']=talib.PLUS_DI(df['High'],df['Low'],df['Close'],timeperiod=30)
df['DI-']=talib.MINUS_DI(df['High'],df['Low'],df['Close'],timeperiod=30)
df['ADX']=talib.ADX(df['High'],df['Low'],df['Close'],timeperiod=30)
df['P&L'] = df['Close'] - df['Open']
df['pnl_close']=df['Close'].diff()

<ipython-input-7-1c11e47b4067>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MA']=talib.SMA(df['Adj Close'],timeperiod=12)
<ipython-input-7-1c11e47b4067>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EMA']=talib.EMA(df['Adj Close'],timeperiod=12)
<ipython-input-7-1c11e47b4067>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [ ]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,MA,EMA,MACD,MACD_Signal,RSI,WMA,Upper Band,Mid Band,Low Band,DI+,DI-,ADX,P&L,pnl_close
Date,,,,,,,,,,,,,,,,,,,,
2015-01-02,27.847500,27.860001,26.837500,27.332500,24.531765,212818400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.514999,NaN
2015-01-05,27.072500,27.162500,26.352501,26.562500,23.840664,257142000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.510000,-0.770000
2015-01-06,26.635000,26.857500,26.157499,26.565001,23.842911,263188400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.070000,0.002501
2015-01-07,26.799999,27.049999,26.674999,26.937500,24.177237,160423600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.137501,0.372499
2015-01-08,27.307501,28.037500,27.174999,27.972500,25.106188,237458000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.664999,1.035000


In [ ]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'MA', 'EMA',
       'MACD', 'MACD_Signal', 'RSI', 'WMA', 'Upper Band', 'Mid Band',
       'Low Band', 'DI+', 'DI-', 'ADX', 'P&L', 'pnl_close'],
      dtype='object')

In [ ]:
df=df.bfill()

###Standard LSTM

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
import plotly.graph_objects as go

data = df
prices = data['Close'].values.reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(0, 1))
prices_normalized = scaler.fit_transform(prices)

train_size = int(len(prices_normalized) * 0.8)
train_data = prices_normalized[:train_size]
test_data = prices_normalized[train_size:]
def create_dataset(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps, 0])
    return np.array(X), np.array(y)
time_steps = 30
X_train, y_train = create_dataset(train_data, time_steps)
X_test, y_test = create_dataset(test_data, time_steps)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
model = Sequential()
model.add(LSTM(units=100, activation='relu', input_shape=(time_steps, 1),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

model.compile(optimizer='adam', loss='mse')


model.fit(X_train, y_train, epochs=10, batch_size=32)
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
x_test_dates = data.index.values[train_size + time_steps:]
actual_prices = data['Close'].values[train_size + time_steps:]

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test_dates, y=actual_prices, mode='lines', name='Actual Prices'))
fig.add_trace(go.Scatter(x=x_test_dates, y=predictions.flatten(), mode='lines', name='Predictions'))
fig.update_layout(title='AAPL Stock Price Predictions', xaxis_title='Date', yaxis_title='Price')
fig.show()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 100)           40800     
                                                                 
 dropout (Dropout)           (None, 30, 100)           0         
                                                                 
 lstm_1 (LSTM)               (None, 30, 50)            30200     
                                                                 
 dropout_1 (Dropout)         (None, 30, 50)            0         
                                                                 
 lstm_2 (LSTM)               (None, 30, 50)            20200     
                                                                 
 dropout_2 (Dropout)         (None, 30, 50)            0         
                                                                 
 lstm_3 (LSTM)               (None, 30, 50)            2

####5-Day prediction

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense
import plotly.graph_objects as go
data = df
data['Date']=data.index
prices = data['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
prices_normalized = scaler.fit_transform(prices)
def create_dataset(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps - 5):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps:i + time_steps + 5, 0])
    return np.array(X), np.array(y)


time_steps = 30


X, y = create_dataset(prices_normalized, time_steps)


X = np.reshape(X, (X.shape[0], X.shape[1], 1))

model = Sequential()
model.add(LSTM(units=100, activation='relu', input_shape=(time_steps, 1),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dense(5))


model.compile(optimizer='adam', loss='mse')


model.fit(X, y, epochs=10, batch_size=32)


last_sequence = prices_normalized[-time_steps:].reshape(1, time_steps, 1)
predicted_prices = model.predict(last_sequence)


predicted_prices = scaler.inverse_transform(predicted_prices)


next_5_days = pd.date_range(start=data['Date'].iloc[-1], periods=5, freq='B').strftime('%Y-%m-%d')


fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Historical Prices'))
fig.add_trace(go.Scatter(x=next_5_days, y=predicted_prices.flatten(), mode='lines', name='Predicted Prices'))
fig.update_layout(title='AAPL Stock Price Predictions for the Next 5 Days', xaxis_title='Date', yaxis_title='Price')
fig.show()


Epoch 1/10
66/66 [==============================] - 9s 74ms/step - loss: 0.0299
Epoch 2/10
66/66 [==============================] - 7s 111ms/step - loss: 0.0012
Epoch 3/10
66/66 [==============================] - 6s 98ms/step - loss: 0.0011
Epoch 4/10
66/66 [==============================] - 7s 108ms/step - loss: 0.0011
Epoch 5/10
66/66 [==============================] - 8s 123ms/step - loss: 0.0011
Epoch 6/10
66/66 [==============================] - 5s 70ms/step - loss: 9.7939e-04
Epoch 7/10
66/66 [==============================] - 3s 49ms/step - loss: 8.9705e-04
Epoch 8/10
66/66 [==============================] - 4s 59ms/step - loss: 9.7314e-04
Epoch 9/10
66/66 [==============================] - 4s 56ms/step - loss: 0.0012
Epoch 10/10
1/1 [==============================] - 1s 811ms/step


###Bidirectional LSTM

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Bidirectional
import plotly.graph_objects as go

data = df
prices = data['Close'].values.reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(0, 1))
prices_normalized = scaler.fit_transform(prices)

train_size = int(len(prices_normalized) * 0.8)
train_data = prices_normalized[:train_size]
test_data = prices_normalized[train_size:]
def create_dataset(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps, 0])
    return np.array(X), np.array(y)
time_steps = 30
X_train, y_train = create_dataset(train_data, time_steps)
X_test, y_test = create_dataset(test_data, time_steps)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
model = Sequential()
model.add(Bidirectional(LSTM(units=100, activation='relu', input_shape=(time_steps, 1),return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(50,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(50,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(50,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(50,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(50)))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')


model.fit(X_train, y_train, epochs=10, batch_size=32)
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
x_test_dates = data.index.values[train_size + time_steps:]
actual_prices = data['Close'].values[train_size + time_steps:]

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test_dates, y=actual_prices, mode='lines', name='Actual Prices'))
fig.add_trace(go.Scatter(x=x_test_dates, y=predictions.flatten(), mode='lines', name='Predictions'))
fig.update_layout(title='AAPL Stock Price Predictions', xaxis_title='Date', yaxis_title='Price')
fig.show()


Epoch 1/10
53/53 [==============================] - 39s 241ms/step - loss: 0.0083
Epoch 2/10
53/53 [==============================] - 13s 241ms/step - loss: 9.5870e-04
Epoch 3/10
53/53 [==============================] - 13s 250ms/step - loss: 7.0607e-04
Epoch 4/10
53/53 [==============================] - 13s 239ms/step - loss: 6.2920e-04
Epoch 5/10
53/53 [==============================] - 13s 237ms/step - loss: 7.6277e-04
Epoch 6/10
53/53 [==============================] - 13s 238ms/step - loss: 6.0501e-04
Epoch 7/10
53/53 [==============================] - 12s 233ms/step - loss: 6.3004e-04
Epoch 8/10
53/53 [==============================] - 12s 230ms/step - loss: 6.5609e-04
Epoch 9/10
53/53 [==============================] - 12s 229ms/step - loss: 5.6443e-04
Epoch 10/10
13/13 [==============================] - 5s 52ms/step


####5-Day prediction

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Bidirectional
import plotly.graph_objects as go
data = df
data['Date']=data.index
prices = data['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
prices_normalized = scaler.fit_transform(prices)
def create_dataset(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps - 5):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps:i + time_steps + 5, 0])
    return np.array(X), np.array(y)


time_steps = 30


X, y = create_dataset(prices_normalized, time_steps)


X = np.reshape(X, (X.shape[0], X.shape[1], 1))

model = Sequential()
model.add(Bidirectional(LSTM(units=100, activation='relu', input_shape=(time_steps, 1),return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(50)))
model.add(Dense(5))


model.compile(optimizer='adam', loss='mse')


model.fit(X, y, epochs=10, batch_size=32)


last_sequence = prices_normalized[-time_steps:].reshape(1, time_steps, 1)
predicted_prices = model.predict(last_sequence)


predicted_prices = scaler.inverse_transform(predicted_prices)


next_5_days = pd.date_range(start=data['Date'].iloc[-1], periods=5, freq='B').strftime('%Y-%m-%d')


fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Historical Prices'))
fig.add_trace(go.Scatter(x=next_5_days, y=predicted_prices.flatten(), mode='lines', name='Predicted Prices'))
fig.update_layout(title='AAPL Stock Price Predictions for the Next 5 Days', xaxis_title='Date', yaxis_title='Price')
fig.show()


Epoch 1/10
66/66 [==============================] - 14s 84ms/step - loss: 0.0234
Epoch 2/10
66/66 [==============================] - 7s 105ms/step - loss: 0.0014
Epoch 3/10
66/66 [==============================] - 6s 84ms/step - loss: 0.0013
Epoch 4/10
66/66 [==============================] - 7s 105ms/step - loss: 0.0012
Epoch 5/10
66/66 [==============================] - 6s 84ms/step - loss: 0.0010
Epoch 6/10
66/66 [==============================] - 7s 103ms/step - loss: 0.0012
Epoch 7/10
66/66 [==============================] - 6s 84ms/step - loss: 0.0011
Epoch 8/10
66/66 [==============================] - 6s 96ms/step - loss: 9.1766e-04
Epoch 9/10
66/66 [==============================] - 6s 91ms/step - loss: 8.5298e-04
Epoch 10/10
1/1 [==============================] - 1s 1s/step


###CNN-LSTM for Close

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
import plotly.graph_objects as go

data = df
prices = data['Close'].values.reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(0, 1))
prices_normalized = scaler.fit_transform(prices)

train_size = int(len(prices_normalized) * 0.8)
train_data = prices_normalized[:train_size]
test_data = prices_normalized[train_size:]
def create_dataset(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps, 0])
    return np.array(X), np.array(y)
time_steps = 30
X_train, y_train = create_dataset(train_data, time_steps)
X_test, y_test = create_dataset(test_data, time_steps)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(time_steps, 1)))
model.add(MaxPooling1D(pool_size=1))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

model.compile(optimizer='adam', loss='mse')


model.fit(X_train, y_train, epochs=10, batch_size=32)
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
x_test_dates = data.index.values[train_size + time_steps:]
actual_prices = data['Close'].values[train_size + time_steps:]

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test_dates, y=actual_prices, mode='lines', name='Actual Prices'))
fig.add_trace(go.Scatter(x=x_test_dates, y=predictions.flatten(), mode='lines', name='Predictions'))
fig.update_layout(title='AAPL Stock Price Predictions', xaxis_title='Date', yaxis_title='Price')
fig.show()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 28, 64)            256       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 28, 64)           0         
 )                                                               
                                                                 
 lstm_16 (LSTM)              (None, 28, 50)            23000     
                                                                 
 dropout_12 (Dropout)        (None, 28, 50)            0         
                                                                 
 lstm_17 (LSTM)              (None, 28, 50)            20200     
                                                                 
 dropout_13 (Dropout)        (None, 28, 50)            0         
                                                      

####5-Day prediction

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense
import plotly.graph_objects as go
data = df
data['Date']=data.index
prices = data['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
prices_normalized = scaler.fit_transform(prices)
def create_dataset(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps - 5):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps:i + time_steps + 5, 0])
    return np.array(X), np.array(y)


time_steps = 30


X, y = create_dataset(prices_normalized, time_steps)


X = np.reshape(X, (X.shape[0], X.shape[1], 1))

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(time_steps, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(50))
model.add(Dense(5))


model.compile(optimizer='adam', loss='mse')


model.fit(X, y, epochs=10, batch_size=32)


last_sequence = prices_normalized[-time_steps:].reshape(1, time_steps, 1)
predicted_prices = model.predict(last_sequence)


predicted_prices = scaler.inverse_transform(predicted_prices)


next_5_days = pd.date_range(start=data['Date'].iloc[-1], periods=5, freq='B').strftime('%Y-%m-%d')


fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Historical Prices'))
fig.add_trace(go.Scatter(x=next_5_days, y=predicted_prices.flatten(), mode='lines', name='Predicted Prices'))
fig.update_layout(title='AAPL Stock Price Predictions for the Next 5 Days', xaxis_title='Date', yaxis_title='Price')
fig.show()


Epoch 1/10
66/66 [==============================] - 3s 13ms/step - loss: 0.0203
Epoch 2/10
66/66 [==============================] - 1s 13ms/step - loss: 9.4685e-04
Epoch 3/10
66/66 [==============================] - 1s 18ms/step - loss: 9.1199e-04
Epoch 4/10
66/66 [==============================] - 1s 19ms/step - loss: 8.6585e-04
Epoch 5/10
66/66 [==============================] - 1s 18ms/step - loss: 8.2636e-04
Epoch 6/10
66/66 [==============================] - 1s 13ms/step - loss: 8.1641e-04
Epoch 7/10
66/66 [==============================] - 1s 13ms/step - loss: 8.2221e-04
Epoch 8/10
66/66 [==============================] - 1s 12ms/step - loss: 8.2449e-04
Epoch 9/10
66/66 [==============================] - 1s 13ms/step - loss: 6.9260e-04
Epoch 10/10
1/1 [==============================] - 0s 453ms/step


##LSTM with technical indicators with 5 day prediction

###Standard LSTM

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error as MAPE

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as ms
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional ,Dropout
import plotly.graph_objects as go
import talib
import yfinance as yf

df = yf.download('AAPL')
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
res = df[~(df.index < '2015-01-01')]
df = res
df['MA'] = talib.SMA(df['Adj Close'], timeperiod=12)
df['EMA'] = talib.EMA(df['Adj Close'], timeperiod=12)
EMA_12 = pd.Series(df['Close'].ewm(span=12, min_periods=12).mean())
EMA_26 = pd.Series(df['Close'].ewm(span=26, min_periods=26).mean())
df['MACD'] = pd.Series(EMA_26 - EMA_12)
df['MACD_Signal'] = pd.Series(df['MACD'].ewm(span=9, min_periods=9).mean())
df['RSI'] = talib.RSI(df['Adj Close'], 10)
df['WMA'] = talib.WMA(df['Adj Close'], timeperiod=14)
df['Upper Band'], df['Mid Band'], df['Low Band'] = talib.BBANDS(df['Adj Close'], timeperiod=50)
df['DI+'] = talib.PLUS_DI(df['High'], df['Low'], df['Close'], timeperiod=30)
df['DI-'] = talib.MINUS_DI(df['High'], df['Low'], df['Close'], timeperiod=30)
df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=30)
df['P&L'] = df['Close'] - df['Open']
df['pnl_close'] = df['Close'].diff()
df['OBV'] = talib.OBV(df['Adj Close'], df['Volume'])
df = df.bfill()
data = df

features = data[['MA', 'EMA', 'MACD', 'RSI', 'WMA',
                 'Upper Band', 'Mid Band', 'Low Band', 'DI+', 'DI-', 'ADX', 'P&L', 'pnl_close', 'OBV']].values
target = data['Adj Close'].values.reshape(-1, 1)

scaler_features = StandardScaler()
features_normalized = scaler_features.fit_transform(features)

scaler_target = StandardScaler()
target_normalized = scaler_target.fit_transform(target)

def create_dataset(features, target, time_steps=1):
    X, y = [], []
    for i in range(len(features) - time_steps - 30):
        X.append(features[i:(i + time_steps)])
        y.append(target[i + time_steps + 30])
    return np.array(X), np.array(y)

time_steps = 30
X, y = create_dataset(features_normalized, target_normalized, time_steps)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

model = Sequential()
model.add(LSTM(64, return_sequences=True,input_shape=(time_steps,14)))
model.add(LSTM(5))
model.add(Dense(25))
model.add(Dense(5))
model.summary()

model.compile(optimizer='adam', loss='mse')

X_train, X_test = X, X
y_train, y_test = y, y

model.fit(X_train, y_train, epochs=20, batch_size=32)

predictions = model.predict(X_test)
predictions = scaler_target.inverse_transform(predictions)

x_test_dates = data.index.values[time_steps + 30:]
actual_prices = data['Adj Close'].values[time_steps + 30:]
print(ms(actual_prices,predictions[:, 0],squared=False))
print("MAPE:",MAPE(actual_prices,predictions[:, 0]))
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test_dates, y=actual_prices, mode='lines', name='Actual Prices'))
fig.add_trace(go.Scatter(x=x_test_dates, y=predictions[:, 0], mode='lines', name='Predictions'))
fig.update_layout(title='AAPL Stock Price Predictions', xaxis_title='Date', yaxis_title='Price')
fig.show()

last_sequence = features_normalized[-time_steps:].reshape(1, time_steps, X.shape[2])
predicted_prices_normalized = model.predict(last_sequence)

predicted_prices = scaler_target.inverse_transform(predicted_prices_normalized)

last_date = data.index[-1]
next_5_days = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=5, freq='B').strftime('%Y-%m-%d')

fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['Adj Close'], mode='lines', name='Historical Prices'))
fig.add_trace(go.Scatter(x=next_5_days, y=predicted_prices.flatten(), mode='lines', name='Predicted Prices'))
fig.update_layout(title='Stock Price Predictions for the Next 5 Days', xaxis_title='Date', yaxis_title='Price')
fig.show()

[*********************100%***********************]  1 of 1 completed


<ipython-input-7-5089c4bc2c5b>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-5089c4bc2c5b>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-5089c4bc2c5b>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-5089c4bc

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 30, 64)            20224     
                                                                 
 lstm_5 (LSTM)               (None, 5)                 1400      
                                                                 
 dense_4 (Dense)             (None, 25)                150       
                                                                 
 dense_5 (Dense)             (None, 5)                 130       
                                                                 
Total params: 21,904
Trainable params: 21,904
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
65/65 [==============================] - 6s 26ms/step - loss: 0.3309
Epoch 2/20
65/65 [==============================] - 3s 40ms/step - loss: 0.0281
Epoch 3/20
65/65 [====

1/1 [==============================] - 0s 56ms/step


###Bi-Directional LSTM

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as ms
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional ,Dropout
import plotly.graph_objects as go
import talib
import yfinance as yf

df = yf.download('AAPL')
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
res = df[~(df.index < '2015-01-01')]
df = res
df['MA'] = talib.SMA(df['Adj Close'], timeperiod=12)
df['EMA'] = talib.EMA(df['Adj Close'], timeperiod=12)
EMA_12 = pd.Series(df['Close'].ewm(span=12, min_periods=12).mean())
EMA_26 = pd.Series(df['Close'].ewm(span=26, min_periods=26).mean())
df['MACD'] = pd.Series(EMA_26 - EMA_12)
df['MACD_Signal'] = pd.Series(df['MACD'].ewm(span=9, min_periods=9).mean())
df['RSI'] = talib.RSI(df['Adj Close'], 10)
df['WMA'] = talib.WMA(df['Adj Close'], timeperiod=14)
df['Upper Band'], df['Mid Band'], df['Low Band'] = talib.BBANDS(df['Adj Close'], timeperiod=50)
df['DI+'] = talib.PLUS_DI(df['High'], df['Low'], df['Close'], timeperiod=30)
df['DI-'] = talib.MINUS_DI(df['High'], df['Low'], df['Close'], timeperiod=30)
df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=30)
df['P&L'] = df['Close'] - df['Open']
df['pnl_close'] = df['Close'].diff()
df['OBV'] = talib.OBV(df['Adj Close'], df['Volume'])
df = df.bfill()
data = df

features = data[['MA', 'EMA', 'MACD', 'RSI', 'WMA',
                 'Upper Band', 'Mid Band', 'Low Band', 'DI+', 'DI-', 'ADX', 'P&L', 'pnl_close', 'OBV']].values
target = data['Adj Close'].values.reshape(-1, 1)

scaler_features = StandardScaler()
features_normalized = scaler_features.fit_transform(features)

scaler_target = StandardScaler()
target_normalized = scaler_target.fit_transform(target)

def create_dataset(features, target, time_steps=1):
    X, y = [], []
    for i in range(len(features) - time_steps - 30):
        X.append(features[i:(i + time_steps)])
        y.append(target[i + time_steps + 30])
    return np.array(X), np.array(y)

time_steps = 30
X, y = create_dataset(features_normalized, target_normalized, time_steps)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

model2 = Sequential()
model2.add(Bidirectional(LSTM(64, return_sequences=True),input_shape=(time_steps,14)))
model2.add(Bidirectional(LSTM(5)))
model2.add(Dense(25))
model2.add(Dense(5))
model2.summary()

model2.compile(optimizer='adam', loss='mse')

X_train, X_test = X, X
y_train, y_test = y, y

model2.fit(X_train, y_train, epochs=20, batch_size=32)

predictions = model2.predict(X_test)
predictions = scaler_target.inverse_transform(predictions)

x_test_dates = data.index.values[time_steps + 30:]
actual_prices = data['Adj Close'].values[time_steps + 30:]
print(ms(actual_prices,predictions[:, 0],squared=False))
print("MAPE:",MAPE(actual_prices,predictions[:, 0]))
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test_dates, y=actual_prices, mode='lines', name='Actual Prices'))
fig.add_trace(go.Scatter(x=x_test_dates, y=predictions[:, 0], mode='lines', name='Predictions'))
fig.update_layout(title='AAPL Stock Price Predictions', xaxis_title='Date', yaxis_title='Price')
fig.show()
last_sequence = features_normalized[-time_steps:].reshape(1, time_steps, X.shape[2])
predicted_prices_normalized = model2.predict(last_sequence)

predicted_prices = scaler_target.inverse_transform(predicted_prices_normalized)

last_date = data.index[-1]
next_5_days = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=5, freq='B').strftime('%Y-%m-%d')

fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['Adj Close'], mode='lines', name='Historical Prices'))
fig.add_trace(go.Scatter(x=next_5_days, y=predicted_prices.flatten(), mode='lines', name='Predicted Prices'))
fig.update_layout(title='Stock Price Predictions for the Next 5 Days', xaxis_title='Date', yaxis_title='Price')
fig.show()

[*********************100%***********************]  1 of 1 completed


<ipython-input-6-0664fa660640>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-0664fa660640>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-0664fa660640>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-0664fa66

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 30, 128)          40448     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 10)               5360      
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 25)                275       
                                                                 
 dense_3 (Dense)             (None, 5)                 130       
                                                                 
Total params: 46,213
Trainable params: 46,213
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
65/65 [==============================] 

1/1 [==============================] - 0s 25ms/step


###CNN-LSTM model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as ms
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Dropout
import plotly.graph_objects as go
import talib
import yfinance as yf

df = yf.download('AAPL')
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
res = df[~(df.index < '2015-01-01')]
df = res
df['MA'] = talib.SMA(df['Adj Close'], timeperiod=12)
df['EMA'] = talib.EMA(df['Adj Close'], timeperiod=12)
EMA_12 = pd.Series(df['Close'].ewm(span=12, min_periods=12).mean())
EMA_26 = pd.Series(df['Close'].ewm(span=26, min_periods=26).mean())
df['MACD'] = pd.Series(EMA_26 - EMA_12)
df['MACD_Signal'] = pd.Series(df['MACD'].ewm(span=9, min_periods=9).mean())
df['RSI'] = talib.RSI(df['Adj Close'], 10)
df['WMA'] = talib.WMA(df['Adj Close'], timeperiod=14)
df['Upper Band'], df['Mid Band'], df['Low Band'] = talib.BBANDS(df['Adj Close'], timeperiod=50)
df['DI+'] = talib.PLUS_DI(df['High'], df['Low'], df['Close'], timeperiod=30)
df['DI-'] = talib.MINUS_DI(df['High'], df['Low'], df['Close'], timeperiod=30)
df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=30)
df['P&L'] = df['Close'] - df['Open']
df['pnl_close'] = df['Close'].diff()
df['OBV'] = talib.OBV(df['Adj Close'], df['Volume'])
df = df.bfill()
data = df

features = data[['MA', 'EMA', 'MACD', 'RSI', 'WMA',
                 'Upper Band', 'Mid Band', 'Low Band', 'DI+', 'DI-', 'ADX', 'P&L', 'pnl_close', 'OBV']].values
target = data['Adj Close'].values.reshape(-1, 1)

scaler_features = StandardScaler()
features_normalized = scaler_features.fit_transform(features)

scaler_target = StandardScaler()
target_normalized = scaler_target.fit_transform(target)

def create_dataset(features, target, time_steps=1):
    X, y = [], []
    for i in range(len(features) - time_steps - 30):
        X.append(features[i:(i + time_steps)])
        y.append(target[i + time_steps + 30])
    return np.array(X), np.array(y)

time_steps = 30
X, y = create_dataset(features_normalized, target_normalized, time_steps)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

model3 = Sequential()
model3.add(Conv1D(filters=64, kernel_size=3, activation='tanh', input_shape=(time_steps, 14)))
model3.add(MaxPooling1D(pool_size=2))
model3.add(LSTM(5,return_sequences=True))
model3.add(LSTM(5))
model3.add(Dense(25))
model3.add(Dense(5))
model3.summary()

model3.compile(optimizer='adam', loss='mse')

X_train, X_test = X, X
y_train, y_test = y, y

model3.fit(X_train, y_train, epochs=20, batch_size=32)

predictions = model3.predict(X_test)
predictions = scaler_target.inverse_transform(predictions)
x_test_dates = data.index.values[time_steps + 30:]
actual_prices = data['Adj Close'].values[time_steps + 30:]
print(ms(actual_prices,predictions[:, 0],squared=False))
print("MAPE:",MAPE(actual_prices,predictions[:, 0]))
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test_dates, y=actual_prices, mode='lines', name='Actual Prices'))
fig.add_trace(go.Scatter(x=x_test_dates, y=predictions[:, 0], mode='lines', name='Predictions'))
fig.update_layout(title='AAPL Stock Price Predictions', xaxis_title='Date', yaxis_title='Price')
fig.show()

last_sequence = features_normalized[-time_steps:].reshape(1, time_steps, X.shape[2])
predicted_prices_normalized = model3.predict(last_sequence)

predicted_prices = scaler_target.inverse_transform(predicted_prices_normalized)

last_date = data.index[-1]
next_5_days = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=5, freq='B').strftime('%Y-%m-%d')

fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['Adj Close'], mode='lines', name='Historical Prices'))
fig.add_trace(go.Scatter(x=next_5_days, y=predicted_prices.flatten(), mode='lines', name='Predicted Prices'))
fig.update_layout(title='Stock Price Predictions for the Next 5 Days', xaxis_title='Date', yaxis_title='Price')
fig.show()

[*********************100%***********************]  1 of 1 completed


<ipython-input-12-dc6c8c09c710>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-dc6c8c09c710>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-dc6c8c09c710>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-dc6c

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 28, 64)            2752      
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 14, 64)           0         
 1D)                                                             
                                                                 
 lstm_11 (LSTM)              (None, 14, 5)             1400      
                                                                 
 lstm_12 (LSTM)              (None, 5)                 220       
                                                                 
 dense_10 (Dense)            (None, 25)                150       
                                                                 
 dense_11 (Dense)            (None, 5)                 130       
                                                      

1/1 [==============================] - 0s 29ms/step


##CNN-LSTM , Xgboost stack model ( meta model )

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as ms
from sklearn.metrics import mean_absolute_percentage_error as mape
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D
import talib
import yfinance as yf
import xgboost as xgb
import plotly.graph_objects as go

df = yf.download('AAPL')
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
res = df[~(df.index < '2015-01-01')]
df = res
df['MA'] = talib.SMA(df['Adj Close'], timeperiod=12)
df['EMA'] = talib.EMA(df['Adj Close'], timeperiod=12)
EMA_12 = pd.Series(df['Close'].ewm(span=12, min_periods=12).mean())
EMA_26 = pd.Series(df['Close'].ewm(span=26, min_periods=26).mean())
df['MACD'] = pd.Series(EMA_26 - EMA_12)
df['MACD_Signal'] = pd.Series(df['MACD'].ewm(span=9, min_periods=9).mean())
df['RSI'] = talib.RSI(df['Adj Close'], 10)
df['WMA'] = talib.WMA(df['Adj Close'], timeperiod=14)
df['Upper Band'], df['Mid Band'], df['Low Band'] = talib.BBANDS(df['Adj Close'], timeperiod=50)
df['DI+'] = talib.PLUS_DI(df['High'], df['Low'], df['Close'], timeperiod=30)
df['DI-'] = talib.MINUS_DI(df['High'], df['Low'], df['Close'], timeperiod=30)
df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=30)
df['P&L'] = df['Close'] - df['Open']
df['pnl_close'] = df['Close'].diff()
df['OBV'] = talib.OBV(df['Adj Close'], df['Volume'])
df = df.bfill()
data = df

features = data[['MA', 'EMA', 'MACD', 'RSI', 'WMA',
                 'Upper Band', 'Mid Band', 'Low Band', 'DI+', 'DI-', 'ADX', 'P&L', 'pnl_close', 'OBV']].values
target = data['Adj Close'].values.reshape(-1, 1)

scaler_features = StandardScaler()
features_normalized = scaler_features.fit_transform(features)

scaler_target = StandardScaler()
target_normalized = scaler_target.fit_transform(target)

def create_dataset(features, target, time_steps=1):
    X, y = [], []
    for i in range(len(features) - time_steps - 30):
        X.append(features[i:(i + time_steps)])
        y.append(target[i + time_steps + 30])
    return np.array(X), np.array(y)

time_steps = 30
X, y = create_dataset(features_normalized, target_normalized, time_steps)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

model3 = Sequential()
model3.add(Conv1D(filters=64, kernel_size=3, activation='tanh', input_shape=(time_steps, 14)))
model3.add(MaxPooling1D(pool_size=2))
model3.add(LSTM(5))
model3.add(Dense(25))
model3.add(Dense(5))
model3.summary()

model3.compile(optimizer='adam', loss='mse')

X_train, X_test = X, X
y_train, y_test = y, y

model3.fit(X_train, y_train, epochs=20, batch_size=32)

lstm_predictions = model3.predict(X_test)
lstm_predictions = scaler_target.inverse_transform(lstm_predictions)
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train.reshape(X_train.shape[0], -1), y_train.ravel())
xgb_predictions = xgb_model.predict(X_test.reshape(X_test.shape[0], -1))


stacked_predictions = np.column_stack((lstm_predictions, xgb_predictions))

meta_model = xgb.XGBRegressor()
meta_model.fit(stacked_predictions, y_test.ravel())

final_predictions = meta_model.predict(stacked_predictions)

predicted_prices = scaler_target.inverse_transform(final_predictions.reshape(-1, 1))

predicted_dates = df.index[time_steps + 30:]
mape_value = mape(predicted_prices, data['Adj Close'][time_steps + 30:].values)
print(f"Mean Absolute Percentage Error (MAPE): {mape_value}")
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=data['Adj Close'], name='Actual'))
fig.add_trace(go.Scatter(x=predicted_dates, y=predicted_prices.flatten(), name='Predicted'))

fig.update_layout(title='Actual vs Predicted Stock Prices',
                  xaxis_title='Date',
                  yaxis_title='Price')

fig.show()


[*********************100%***********************]  1 of 1 completed


<ipython-input-52-a603e3f80e55>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-52-a603e3f80e55>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-52-a603e3f80e55>:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-52-a603

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_19 (Conv1D)          (None, 28, 64)            2752      
                                                                 
 max_pooling1d_19 (MaxPoolin  (None, 14, 64)           0         
 g1D)                                                            
                                                                 
 lstm_45 (LSTM)              (None, 5)                 1400      
                                                                 
 dense_49 (Dense)            (None, 25)                150       
                                                                 
 dense_50 (Dense)            (None, 5)                 130       
                                                                 
Total params: 4,432
Trainable params: 4,432
Non-trainable params: 0
___________________________________________________

In [ ]:
# import pandas as pd
# index = df.index[:len(predicted_prices)]
# df_result = pd.DataFrame({'Actual': data['Adj Close'].values.flatten()[:len(index)], 'Predicted': predicted_prices.flatten()}, index=index)
# df_result

In [ ]:
last_sequence = features_normalized[-time_steps:].reshape(1, time_steps, X.shape[2])
predicted_prices_normalized = model3.predict(last_sequence)

predicted_prices = scaler_target.inverse_transform(predicted_prices_normalized)

last_date = data.index[-1]
next_5_days = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=5, freq='B').strftime('%Y-%m-%d')

fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['Adj Close'], mode='lines', name='Historical Prices'))
fig.add_trace(go.Scatter(x=next_5_days, y=predicted_prices.flatten(), mode='lines', name='Predicted Prices'))
fig.update_layout(title='Stock Price Predictions for the Next 5 Days', xaxis_title='Date', yaxis_title='Price')
fig.show()

1/1 [==============================] - 0s 25ms/step


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error as mape
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D
import talib
import yfinance as yf
import xgboost as xgb
import plotly.graph_objects as go

df = yf.download('AAPL')
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
res = df[~(df.index < '2015-01-01')]
df = res
df['MA'] = talib.SMA(df['Adj Close'], timeperiod=12)
df['EMA'] = talib.EMA(df['Adj Close'], timeperiod=12)
EMA_12 = pd.Series(df['Close'].ewm(span=12, min_periods=12).mean())
EMA_26 = pd.Series(df['Close'].ewm(span=26, min_periods=26).mean())
df['MACD'] = pd.Series(EMA_26 - EMA_12)
df['MACD_Signal'] = pd.Series(df['MACD'].ewm(span=9, min_periods=9).mean())
df['RSI'] = talib.RSI(df['Adj Close'], 10)
df['WMA'] = talib.WMA(df['Adj Close'], timeperiod=14)
df['Upper Band'], df['Mid Band'], df['Low Band'] = talib.BBANDS(df['Adj Close'], timeperiod=50)
df['DI+'] = talib.PLUS_DI(df['High'], df['Low'], df['Close'], timeperiod=30)
df['DI-'] = talib.MINUS_DI(df['High'], df['Low'], df['Close'], timeperiod=30)
df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=30)
df['P&L'] = df['Close'] - df['Open']
df['pnl_close'] = df['Close'].diff()
df['OBV'] = talib.OBV(df['Adj Close'], df['Volume'])
df = df.bfill()
data = df

features = data[['MA', 'EMA', 'MACD', 'RSI', 'WMA',
                 'Upper Band', 'Mid Band', 'Low Band', 'DI+', 'DI-', 'ADX', 'P&L', 'pnl_close', 'OBV']].values
target = data['Adj Close'].values.reshape(-1, 1)

scaler_features = StandardScaler()
features_normalized = scaler_features.fit_transform(features)

scaler_target = StandardScaler()
target_normalized = scaler_target.fit_transform(target)

def create_dataset(features, target, time_steps=1):
    X, y = [], []
    for i in range(len(features) - time_steps - 30):
        X.append(features[i:(i + time_steps)])
        y.append(target[i + time_steps + 30])
    return np.array(X), np.array(y)

time_steps = 30
X, y = create_dataset(features_normalized, target_normalized, time_steps)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

model3 = Sequential()
model3.add(Conv1D(filters=64, kernel_size=3, activation='tanh', input_shape=(time_steps, 14)))
model3.add(MaxPooling1D(pool_size=2))
model3.add(LSTM(5))
model3.add(Dense(25))
model3.add(Dense(5))
model3.summary()

model3.compile(optimizer='adam', loss='mse')

test_size = int(len(X) * 0.2)
X_train, X_test = X[:-test_size], X[-test_size:]
y_train, y_test = y[:-test_size], y[-test_size:]

model3.fit(X_train, y_train, epochs=20, batch_size=32)

lstm_predictions = model3.predict(X_test)
lstm_predictions = scaler_target.inverse_transform(lstm_predictions)
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train.reshape(X_train.shape[0], -1), y_train.ravel())
xgb_predictions = xgb_model.predict(X_test.reshape(X_test.shape[0], -1))

stacked_predictions = np.column_stack((lstm_predictions, xgb_predictions))

meta_model = xgb.XGBRegressor()
meta_model.fit(stacked_predictions, y_test.ravel())

final_predictions = meta_model.predict(stacked_predictions)

predicted_prices = scaler_target.inverse_transform(final_predictions.reshape(-1, 1))
predicted_dates = df.index[-test_size - time_steps + 30:]

mape_value = mape(predicted_prices, data['Adj Close'].values[-test_size - time_steps + 30:])

print(f"Mean Absolute Percentage Error (MAPE): {mape_value}")
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=data['Adj Close'], name='Actual'))
fig.add_trace(go.Scatter(x=predicted_dates, y=predicted_prices.flatten(), name='Predicted'))

fig.update_layout(title='Actual vs Predicted Stock Prices',
                  xaxis_title='Date',
                  yaxis_title='Price')

fig.show()


[*********************100%***********************]  1 of 1 completed


<ipython-input-76-ec6aa31994cf>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-76-ec6aa31994cf>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-76-ec6aa31994cf>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-76-ec6a

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_27 (Conv1D)          (None, 28, 64)            2752      
                                                                 
 max_pooling1d_27 (MaxPoolin  (None, 14, 64)           0         
 g1D)                                                            
                                                                 
 lstm_53 (LSTM)              (None, 5)                 1400      
                                                                 
 dense_65 (Dense)            (None, 25)                150       
                                                                 
 dense_66 (Dense)            (None, 5)                 130       
                                                                 
Total params: 4,432
Trainable params: 4,432
Non-trainable params: 0
___________________________________________________